# AutoModelizer
---

La idea de este proyecto es encontrar el mejor modelo de CNN que se adapte al dataset correspondiente, para ello usando algoritmos evolutivos. Este tipo de soluciones se conocen como neuroevoluciones

A continuación un ejemplo básico de como funcionan este tipo de algoritmos

In [67]:
import numpy as np

def fitness(x):
    return x ** 2

population_size = 10
population = np.random.uniform(-10, 10, population_size)

In [68]:
def select_parent_tournament(population, scores, k=3):
    selection_ix = np.random.randint(len(population), size=k)
    selected = population[selection_ix]
    ix = np.argmax(scores[selection_ix])
    return selected[ix]

def crossover(p1, p2):
    child = (p1 + p2) / 2
    return child

def mutate(x):
    mutation_chance = 0.1
    if np.random.rand() < mutation_chance:
        x += np.random.uniform(-1, 1)
    return x

In [69]:
n_generations = 200

for generation in range(n_generations):
    scores = np.array([fitness(x) for x in population])
    new_population = []
    for _ in range(population_size):
        parent1 = select_parent_tournament(population, scores)
        parent2 = select_parent_tournament(population, scores)
        child = crossover(parent1, parent2)
        child = mutate(child)
        new_population.append(child)
    population = np.array(new_population)
    best_score = np.max(scores)
    print(f"Generación {generation}, x = {child} Mejor puntuación {best_score}")

best_solution = population[np.argmax(scores)]
print(f"Mejor solución: x = {best_solution}, f(x) = {fitness(best_solution)}")


Generación 0, x = -5.888127550033035 Mejor puntuación 90.09681734402889
Generación 1, x = -9.569402820633517 Mejor puntuación 90.09681734402889
Generación 2, x = -9.317787290792198 Mejor puntuación 91.5734703435487
Generación 3, x = -9.569402820633517 Mejor puntuación 93.76889839491378
Generación 4, x = -9.907184843028645 Mejor puntuación 102.63585331086406
Generación 5, x = -9.907184843028645 Mejor puntuación 102.63585331086406
Generación 6, x = -10.339887853823189 Mejor puntuación 114.63220007061057
Generación 7, x = -10.655988965509724 Mejor puntuación 114.63220007061057
Generación 8, x = -10.635627195425519 Mejor puntuación 113.550100833065
Generación 9, x = -10.64580808046762 Mejor puntuación 113.550100833065
Generación 10, x = -10.655988965509724 Mejor puntuación 115.60369336232455
Generación 11, x = -11.36202952366327 Mejor puntuación 115.60369336232455
Generación 12, x = -11.079391093357756 Mejor puntuación 130.11660077283054
Generación 13, x = -11.220710308510514 Mejor puntuac

Preparamos el dataset de prueba

In [70]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preparar los DataLoader para los conjuntos de entrenamiento y validación
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)


comprobamos si tenemos acceso a la GPU


In [71]:
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible en tu sistema.")
else:
    print("CUDA (GPU) no está disponible en tu sistema.")

CUDA (GPU) está disponible en tu sistema.


Generamos un par de arquitecturas de prueba

In [72]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    


Construimos una función que sea capaz de crear modelos en base a vectores que representen la arquitectura de la red.
De este modo el algorimo evolutivo puede ir adaptando y cambiando la red fácilmente

In [73]:
import torch

def build_cnn_from_individual(individual):
    layers = []
    num_layers = individual['num_conv_layers']
    out_channels_previous_layer = 1  # Imagen de entrada en escala de grises (1 canal para MNIST)

    for i in range(num_layers):
        out_channels = individual['filters'][i]
        kernel_size = individual['filter_sizes'][i]
        
        conv_layer = nn.Conv2d(out_channels_previous_layer, out_channels, kernel_size=kernel_size, padding=1)
        layers.append(conv_layer)
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

        out_channels_previous_layer = out_channels

    # Temporalmente crear un modelo para calcular el tamaño de salida de las capas convolucionales
    temp_model = nn.Sequential(*layers)

    # Calcular el tamaño de salida usando un tensor dummy
    dummy_input = torch.zeros(1, 1, 28, 28)  # Tamaño de entrada para MNIST
    output_size = temp_model(dummy_input).view(-1).shape[0]

    # Ahora, sabiendo el tamaño de salida, podemos definir las capas lineales correctamente
    layers.append(nn.Flatten())
    layers.append(nn.Linear(output_size, 10))  # Salida de 10 clases para MNIST
    print(*layers)
    return nn.Sequential(*layers)


population_size = 10
min_conv_layers = 1
max_conv_layers = 3
min_filters = 16
max_filters = 128
filter_sizes = [3, 5]




Definimos la población inicial

In [74]:
import random
from tqdm import tqdm

def generate_individual(min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes):
    individual = {
        'num_conv_layers': random.randint(min_conv_layers, max_conv_layers),
        'filters': [],
        'filter_sizes': [],
    }

    for _ in range(individual['num_conv_layers']):
        individual['filters'].append(random.randint(min_filters, max_filters))
        individual['filter_sizes'].append(random.choice(filter_sizes))
    
    # Agrega más parámetros según sea necesario, como capas completamente conectadas, etc.

    return individual

def initialize_population(pop_size, min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes):
    return [generate_individual(min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes) for _ in range(pop_size)]

population = initialize_population(population_size, min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes)

# Construir el modelo para el primer individuo de la población
model = build_cnn_from_individual(population[0])
print(model)



Sequential(
  (0): Conv2d(1, 108, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): Linear(in_features=21168, out_features=10, bias=True)
)


In [75]:
# Ejemplo de inicialización de la población
population_size = 10
min_conv_layers = 1
max_conv_layers = 3
min_filters = 16
max_filters = 128
filter_sizes = [3, 5]

population = initialize_population(population_size, min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes)

epochs = 10

population


[{'num_conv_layers': 2, 'filters': [114, 87], 'filter_sizes': [3, 5]},
 {'num_conv_layers': 2, 'filters': [46, 109], 'filter_sizes': [5, 5]},
 {'num_conv_layers': 3, 'filters': [40, 43, 122], 'filter_sizes': [3, 3, 5]},
 {'num_conv_layers': 2, 'filters': [32, 120], 'filter_sizes': [3, 3]},
 {'num_conv_layers': 2, 'filters': [69, 49], 'filter_sizes': [3, 3]},
 {'num_conv_layers': 1, 'filters': [72], 'filter_sizes': [5]},
 {'num_conv_layers': 2, 'filters': [128, 65], 'filter_sizes': [5, 3]},
 {'num_conv_layers': 3, 'filters': [27, 26, 75], 'filter_sizes': [3, 3, 3]},
 {'num_conv_layers': 3, 'filters': [66, 127, 91], 'filter_sizes': [3, 5, 3]},
 {'num_conv_layers': 3, 'filters': [96, 62, 20], 'filter_sizes': [3, 3, 5]}]

definimos el entrenamiento y testeo

In [76]:
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

def evaluate_individual(individual, train_loader, val_loader, device='cuda', epochs=5):
    # Construir el modelo basado en el individuo
    model = build_cnn_from_individual(individual).to(device)
    
    # Definir el optimizador y la función de pérdida
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()

    # Entrenamiento
    for epoch in range(epochs):
        model.train()
        progress_bar = tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{epochs}', unit='batch')
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            # Actualizar la barra de progreso con la última información de pérdida
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item())})
            progress_bar.update()  # Forzar la actualización de la barra de progreso
            
        progress_bar.close()  # Cerrar la barra de progreso al final de cada época

    # Evaluación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = correct / total
    return accuracy  # Esta es la "aptitud" del individuo


In [77]:
def evaluate_population(population, train_loader, val_loader, device):
    fitness_scores = []
    
    for individual in population:
        fitness = evaluate_individual(individual, train_loader, val_loader, device)
        fitness_scores.append(fitness)
    
    return fitness_scores

# Asumiendo que 'population' ya ha sido inicializada
fitness_scores = evaluate_population(population, train_loader, val_loader, device)

# Opcional: Almacenar los individuos y sus puntuaciones en una lista de tuplas y ordenarlos
population_with_scores = list(zip(population, fitness_scores))
population_with_scores.sort(key=lambda x: x[1], reverse=True)  # Ordena de mayor a menor aptitud

# Imprime los resultados
for i, (individual, score) in enumerate(population_with_scores):
    print(f"Individuo {i}: Aptitud = {score}")
